# Maryland Earned Income Tax Credit

Maryland's Earned Income Tax Credit (EITC) matches a percentage of the federal EITC depending on income and family structure.
In 2021, single childless filers received 100% of the federal EITC up to \$530, while other filers received 50% as a non-refundable credit and 45% as a refundable credit.

## Examples

Consider a set of Maryland households whose income is solely from earnings.

In [1]:
from policyengine_us import IndividualSim
import pandas as pd
import plotly.express as px


def make(adults, children):
    sim = IndividualSim(year=2022)
    sim.add_person(name="head", age=25, is_tax_unit_head=True)
    members = ["head"]
    if adults > 1:
        sim.add_person(name="spouse", is_tax_unit_head=False)
        members += ["spouse"]
    for i in range(children):
        child = "child{}".format(i)
        sim.add_person(name=child, age=5)
        members += [child]
    sim.add_tax_unit(
        name="tax_unit",
        members=members,
    )
    sim.add_spm_unit(name="spm_unit", members=members)
    sim.add_household(name="household", members=members, state_code="MD")
    sim.vary("employment_income", max=60_000, step=100)
    return pd.DataFrame(
        dict(
            employment_income=sim.calc("employment_income")[0],
            eitc=sim.calc("md_eitc")[0].round(),
            eitc_mtr=-sim.deriv(
                "md_eitc",
                "employment_income",
                wrt_target="head",
            ),
            adults=adults,
            children=children,
        )
    )


l = []
for adults in range(1, 3):
    for children in range(0, 3):
        l.append(make(adults, children))

df = pd.concat(l)

LABELS = dict(
    employment_income="Employment income",
    eitc="MD EITC",
    eitc_mtr="MD EITC marginal tax rate",
    adults="Adults",
    children="Children",
)

fig = px.line(
    df,
    "employment_income",
    "eitc",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Maryland EITC",
)
fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat="$,",
    yaxis_range=[0, df.eitc.max() * 1.05],
)
fig.show()

The Maryland Earned Income Tax Credit creates marginal tax rates ranging from -18% to +10.5%, depending on filers' income and family structure.

In [2]:
fig = px.line(
    df,
    "employment_income",
    "eitc_mtr",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Maryland EITC marginal tax rate",
)
fig.update_layout(
    xaxis_tickformat="$,", yaxis_tickformat=".1%", yaxis_range=[-0.15, 0.15]
)
fig.show()

## Budgetary impact

Applying 2022 rules to the 2020 Current Population Survey, OpenFisca US estimates that the Maryland EITC costs \$271 million.

In [3]:
from policyengine_us import Microsimulation

sim = Microsimulation(year=2020)

sim.calc("md_eitc", period=2022).sum() / 1e6

270.83883112002434